# EM0122

## Imports

In [1]:
from ExperimentManagement.ExperimentHierarchy import ExperimentData
import numpy as np

## Generation

In [1]:
#

## Load Hierarchy

In [2]:
EH = ExperimentData.loadHierarchy("D:\\EM0122")

Loading Experimental Hierarchy...
Finished.
Activating auto-logging. Current session state plus future input saved.
Filename       : D:\EM0122\log_file.log
Mode           : append
Output logging : True
Raw input log  : True
Timestamping   : True
State          : active
Logging Initiated


## Image PreProcessing

Imports

In [3]:
from ImagingAnalysis.PreprocessingImages import PreProcessing

Define Directories

In [4]:
RawVideoDirectory = EH.Encoding.folder_dictionary.get("raw_imaging_data")
OutputDirectory = EH.Encoding.folder_dictionary.get("compiled_imaging_data_folder").path

Import MetaData

In [5]:
EH.Encoding.loadBrukerMetaData()

Repackage bruker files into a single tiff

In [5]:
PreProcessing.repackageBrukerTiffs(RawVideoDirectory, OutputDirectory)
EH.Encoding.folder_dictionary.get("compiled_imaging_data_folder").reIndex()

Repackaging...


Loading Images...:   0%|          | 0/7000 [00:00<?, ?it/s]

Loading Images...:   0%|          | 0/7000 [00:00<?, ?it/s]

Loading Images...:   0%|          | 0/7000 [00:00<?, ?it/s]

Loading Images...:   0%|          | 0/7000 [00:00<?, ?it/s]

...

Loading Images...:   0%|          | 0/7000 [00:00<?, ?it/s]

Loading Images...:   0%|          | 0/7000 [00:00<?, ?it/s]

Loading Images...:   0%|          | 0/7000 [00:00<?, ?it/s]

Loading Images...:   0%|          | 0/2895 [00:00<?, ?it/s]

Finished Repackaging Bruker Tiffs


Load Images

In [8]:
images = PreProcessing.loadAllTiffs(OutputDirectory)

Loading Images...:   0%|          | 0/9 [00:00<?, ?it/s]

Filter Images

In [9]:
images = PreProcessing.blockwiseFastFilterTiff(images, Footprint=np.ones((7, 3, 3)))

Filtering Images...:   0%|          | 0/3 [00:00<?, ?it/s]

Remove Shutter Artifact

In [10]:
images = PreProcessing.removeShuttleArtifact(images, chunk_size=7000, artifact_length=1000)

Save Images as Binary

In [11]:
PreProcessing.saveRawBinary(images, OutputDirectory)
EH.Encoding.folder_dictionary.get("compiled_imaging_data_folder").reIndex()
EH.recordMod("Repackaged, filtered, and exported images as raw binary. Made video even length this time")

Finished saving video as a binary.


## Image Processing

Construct Folder for Data

In [4]:
EH.Encoding.addImageSamplingFolder(30)

Motion Correction

In [ ]:
from ImagingAnalysis.Suite2PAnalysis import Suite2PModule

In [5]:
MotionCorrection = Suite2PModule(EH.Encoding.folder_dictionary.get("compiled_imaging_data_folder").path, EH.Encoding.Imaging_30Hz.path, file_type="binary")
MotionCorrection.motionCorrect()
MotionCorrection.exportCroppedCorrection(MotionCorrection.ops)
del MotionCorrection # Clean Up

Reference frame, 32.47 sec.
Registered 7000/56000 in 51.38s
Registered 14000/56000 in 103.20s
Registered 21000/56000 in 154.99s
Registered 28000/56000 in 206.90s
Registered 35000/56000 in 261.91s
Registered 42000/56000 in 317.82s
Registered 49000/56000 in 373.88s
Registered 56000/56000 in 429.61s
Exported Cropped Motion-Corrected Video


Denoise

In [3]:
from ImagingAnalysis.Denoising import DenoisingModule

In [4]:
Denoiser = DenoisingModule("ModelForPyTorch", "binary_video",
                    model_path="C:\\ProgramData\\Anaconda3\\envs\\Calcium-Imaging-Analysis-Pipeline\\pth",
                    data_path="".join([EH.Encoding.Imaging_30Hz.path, "\\suite2p\\plane0"]),
                    output_path="".join([EH.Encoding.Imaging_30Hz.path, "\\denoised"]),
                    image_type="binary",
                    length="21000",
                    workers=4,
                    vram=24,
                    batch_size2=4)
Denoiser.runDenoising()
Denoiser = None

ROI Detection & Classification

In [ ]:
from ImagingAnalysis.Suite2PAnalysis import Suite2PModule

In [ ]:
S2P = Suite2PModule("".join([EH.Encoding.Imaging_30Hz.path, "\\denoised"]), EH.Encoding.Imaging_30Hz.path, file_type="binary")
S2P.roiDetection()
S2P.extractTraces()
S2P.classifyROIs()
S2P.spikeExtraction() # Finalize (Required spks.npy to use GUI)
# Derp
S2P.ops["meanImg"] = S2P.ops.get("max_proj")
S2P.ops["xrange"]= [0, 496]
S2P.ops["yrange"]=[0, 500]

In [ ]:
S2P.save_files()
EH.Encoding.recordMod()
EH.recordMod("S2P Encoding")
EH.saveHierarchy()
del S2P

Fissa: Signal Extraction

In [ ]:
from ImagingAnalysis.FissaAnalysis import FissaModule

In [ ]:
Fissa = FissaModule(data_folder="".join([EH.Encoding.Imaging_30Hz.path]), video_folder="".join([EH.Encoding.Imaging_30Hz.path, "\\denoised"]))
Fissa.initializeFissa()

In [ ]:
Fissa.extractTraces() # simple, call to extract raw traces from videos
Fissa.saveFissaPrep()

Trace Processing

In [ ]:
from ImagingAnalysis.StaticProcessing import Processing

In [ ]:
# let's smooth the data with edge-preserving to make it nicer
Fissa.ProcessedTraces.smoothed_raw = Processing.smoothTraces_TiffOrg(Fissa.preparation.raw, niter=50, kappa=150, gamma=0.15)[0]
Fissa.preparation.raw = Fissa.ProcessedTraces.smoothed_raw.copy()
#Let's use for separation, so replace the raws with smooths
Fissa.passPrepToFissa()

Fissa: Source-Separation

In [ ]:
Fissa.separateTraces() # simple, call to separate the traces
Fissa.saveFissaSep()

Trace Post-Processing

In [ ]:
# Calculate Fo/F
Fissa.ProcessedTraces.dFoF_result = Processing.calculate_dFoF(Fissa.experiment.result, Fissa.frame_rate, raw=Fissa.preparation.raw, merge_after=False)

# Condense the ROI Traces for each Trial into a Single Matrix
Fissa.ProcessedTraces.merged_dFoF_result = Processing.mergeTraces(Fissa.ProcessedTraces.dFoF_result)

# Detrend the Traces by fitting a 4th-order polynomial and subsequently subtracting
Fissa.ProcessedTraces.detrended_merged_dFoF_result = Processing.detrendTraces(Fissa.ProcessedTraces.merged_dFoF_result, order=4, plot=False)

# Save
Fissa.saveProcessedTraces()
EH.recordMod("Encoding Source-Separation")
EH.saveHierarchy()

Cascade: Event/Spike/Firing Rate Inference

In [1]:
from ImagingAnalysis.CascadeAnalysis import CascadeModule

KeyboardInterrupt: 

In [ ]:
Cascade = CascadeModule(Fissa.ProcessedTraces.detrended_merged_dFoF_result, Fissa.frame_rate, model_folder="C:\\ProgramData\\Anaconda3\\envs\\Calcium-Imaging-Analysis-Pipeline\\Pretrained_models")

In [ ]:
# Pull Available Models
list_of_models = Cascade.pullModels(Cascade.model_folder)
# Select Model: If you know what model you want, you should use the string instead.
# This model is Global_EXC_10Hz_smoothing_100ms
# Cascade.model_name = list_of_models[21]
Cascade.model_name = "Global_EXC_30Hz_smoothing100ms"
Cascade.downloadModel(Cascade.model_name, "C:\\ProgramData\\Anaconda3\\envs\\Calcium-Imaging-Analysis-Pipeline\\Pretrained_models")

In [ ]:
# Infer Spike Probability
Cascade.predictSpikeProb() # Simple, call to infer spike probability for each frame
# Calculate Firing Rates # Simple, firing rate = spike probability * imaging frequency
Cascade.ProcessedInferences.firing_rates = Processing.calculateFiringRate(Cascade.spike_prob, Cascade.frame_rate)

In [ ]:
Cascade.saveSpikeProb(Fissa.output_folder)
Cascade.saveProcessedInferences(Fissa.output_folder)

In [ ]:
Cascade.inferDiscreteSpikes()

In [ ]:
Cascade.saveSpikeInference(Fissa.output_folder)

In [ ]:
EH.recordMod("Encoding Cascade")
EH.saveHierarchy()